In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
import tensorflow as tf
import tensorflow.contrib.eager as tfe
print("TensorFlow version: {}".format(tf.VERSION))

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

C:\Users\WEIL\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow version: 1.8.0


## Prepare the Iris data 

In [35]:
# import Iris data
iris = datasets.load_iris()
# only perform a binary classification demo, so only use the first 100 records which only have two types of labels
X = pd.DataFrame(iris.data[:100, :], columns=[ 'Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width'])
print(X.shape)
Y = pd.DataFrame(iris.target[:100], columns=['target'])
print(Y.shape)

(100, 4)
(100, 1)


In [36]:
X.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [37]:
Y.target.unique()

array([0, 1], dtype=int64)

In [38]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

## Prepare Feature Columns

In [40]:
# categorical values with known levels
# gender = tf.feature_column.categorical_column_with_vocabulary_list('gender', ['Female', 'Male'])
# categorical values with unkown levels (many levels)
# gender = tf.feature_column.categorical_column_with_hash_bucket('gender', hash_bucket_size=1000)

# continuous numeric values
Sepal_Length = tf.feature_column.numeric_column('Sepal_Length')
Sepal_Width = tf.feature_column.numeric_column('Sepal_Width')
Petal_Length = tf.feature_column.numeric_column('Petal_Length')
Petal_Width = tf.feature_column.numeric_column('Petal_Width')

In [41]:
feat_cols = [Sepal_Length, Sepal_Width, Petal_Length, Petal_Width]

## Create Input Function

In [42]:
input_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_train, y=y_train, batch_size=30, num_epochs=None, shuffle=True)

## Build the model

Will use the Tensorflow API called estimator.LinearClassifier

In [43]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\WEIL\\AppData\\Local\\Temp\\tmp60e_wqdw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000019050A50470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
model.train(input_fn = input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\WEIL\AppData\Local\Temp\tmp60e_wqdw\model.ckpt.
INFO:tensorflow:loss = 20.794415, step = 1
INFO:tensorflow:global_step/sec: 283.465
INFO:tensorflow:loss = 1.5492458, step = 101 (0.368 sec)
INFO:tensorflow:global_step/sec: 393.795
INFO:tensorflow:loss = 0.8589317, step = 201 (0.244 sec)
INFO:tensorflow:global_step/sec: 424.403
INFO:tensorflow:loss = 0.53586364, step = 301 (0.230 sec)
INFO:tensorflow:global_step/sec: 405.331
INFO:tensorflow:loss = 0.53836036, step = 401 (0.247 sec)
INFO:tensorflow:global_step/sec: 327.388
INFO:tensorflow:loss = 0.66726863, step = 501 (0.317 sec)
INFO:tensorflow:global_step/sec: 449.001
INFO:tensorflow:loss = 0.22947341, step = 601 (0.226 sec)
INFO:tensorflow:global_st

## Evaluation

In [47]:
pred_fn = tf.estimator.inputs.pandas_input_fn(
    x=x_test, batch_size=len(x_test), shuffle=False)

In [48]:
pred = list(model.predict(input_fn = pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\WEIL\AppData\Local\Temp\tmp60e_wqdw\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [49]:
pred[0]

{'class_ids': array([1], dtype=int64),
 'classes': array([b'1'], dtype=object),
 'logistic': array([0.9921727], dtype=float32),
 'logits': array([4.84228], dtype=float32),
 'probabilities': array([0.0078273, 0.9921727], dtype=float32)}

In [59]:
# get the predicted labels
predictions = [p['class_ids'][0] for p in pred]
print(len(predictions))

30


In [63]:
check_predictions = y_test.copy()
check_predictions['pred'] = predictions

In [64]:
check_predictions

,target,pred
80,1,1
84,1,1
33,0,0
81,1,1
93,1,1
17,0,0
36,0,0
82,1,1
69,1,1
65,1,1


In [60]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        14
          1       1.00      1.00      1.00        16

avg / total       1.00      1.00      1.00        30

